In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json

In [2]:
from glob import glob

files = glob('url/*.json')
len(files)

30596

In [3]:
urls = []
for f in files:
    with open(f) as fopen:
        urls.extend(json.load(fopen))
        
urls = sorted(list(set(urls)))
len(urls)

585139

In [4]:
def crawl(url, filename):
    if os.path.exists(filename):
        return
    while True:
        try:
            r = requests.get(url)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    title = soup.find('h1', {'class': 'article-title'}).text
    p = [p_.text for p_ in soup.find_all('p')][2:]
    data = {'title': title, 'p': p, 'url': url}
    with open(filename, 'w') as fopen:
        json.dump(data, fopen)
    return data

In [5]:
directory = 'page'
!mkdir {directory}

mkdir: cannot create directory ‘page’: File exists


In [6]:
from concurrent.futures import ThreadPoolExecutor, as_completed

max_worker = 10

for i in tqdm(range(0, len(urls), max_worker)):
    aranged = np.arange(i, i + max_worker)
    urls_ = [(urls[a], os.path.join(directory, f'{a}.json')) for a in aranged]
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(crawl, url[0], url[1]): url for url in urls_}

    for future in as_completed(futures):
        future.result()

100%|████████████████████████████████████▉| 58513/58514 [46:39<00:00, 20.90it/s]


IndexError: list index out of range